In [1]:
import os

import pandas as pd
import numpy as np
import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import torch.optim as optim
from torch.cuda.amp import GradScaler, autocast
from torch.optim.lr_scheduler import ReduceLROnPlateau
from tqdm import tqdm

print(f"Pytorch version: {torch.__version__}")
print(f"Albumentations version: {A.__version__}")

/mnt/d/AI_and_Datascience/Project/brain-turmor-hierarchical-classification/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Pytorch version: 2.9.1+cu128
Albumentations version: 2.0.8


In [2]:
if torch.cuda.is_available():
    gpu_index = torch.cuda.current_device()
    gpu_name = torch.cuda.get_device_name(gpu_index)
    print(f"GPU khả dụng.")
    print(f"Pytorch đang sử dụng: GPU {gpu_index} - {gpu_name}")
    DEVICE = torch.device(f"cuda:{gpu_index}")
else:
    print("Cảnh báo: không tìm thấy GPU nào khả dụng.")
    print("Sẽ sử dụng CPU.")
    DEVICE = torch.device("cpu")

GPU khả dụng.
Pytorch đang sử dụng: GPU 0 - NVIDIA GeForce RTX 3050 Laptop GPU


In [3]:
BASE_DIR = '../data/brisc2025/'
TRAIN_MANIFEST = os.path.join(BASE_DIR, 'train_clean.csv')
TEST_MANIFEST = os.path.join(BASE_DIR, 'test_clean.csv')
MODEL_SAVE_PATH = '../models/'

BATCH_SIZE = 8
EPOCHS = 60
LEARNING_RATE = 1e-3
NUM_CLASSES = 4

IMG_HEIGHT = 256
IMG_WIDTH = 256
MEAN = [0.485, 0.456, 0.406]
STD = [0.229, 0.224, 0.225]

In [4]:
train_transforms = A.Compose([
    A.Resize(height=IMG_HEIGHT, width=IMG_WIDTH, p=1.0),
    A.HorizontalFlip(p=0.5),
    A.Rotate(limit=15, p=0.3),
    A.Normalize(mean=MEAN, std=STD, p=1.0),
    ToTensorV2(p=1.0)
])

val_transforms = A.Compose([
    A.Resize(height=IMG_HEIGHT, width=IMG_WIDTH, p=1.0),
    A.Normalize(mean=MEAN, std=STD, p=1.0),
    ToTensorV2(p=1.0)
])

print("Đã định nghĩa xong các pipelines tiền xử lý.")

Đã định nghĩa xong các pipelines tiền xử lý.


In [5]:
class BrainMTLDataset(Dataset):
    def __init__(self, manifest_file, base_dir, transform=None):
        self.df = pd.read_csv(manifest_file)
        self.base_dir = base_dir
        self.transform = transform        
        self.labels_map = {
            'no_tumor': 0,
            'glioma': 1,
            'meningioma': 2,
            'pituitary': 3
        }
        print(f"Đã tải {manifest_file} với {len(self.df)} mẫu.")
        print(f"Ánh xạ lớp: {self.labels_map}")

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        class_label = self.labels_map[row['tumor_label']]
        
        raw_img_path = row['relative_path']
        normalized_img_path = raw_img_path.replace("\\", "/")
        img_path = os.path.join(self.base_dir, normalized_img_path)
        image = cv2.imread(img_path)
        if image is None:
            print(f"LỖI NGHIÊM TRỌNG: Vẫn không thể đọc ảnh tại: {img_path}")
            image = np.zeros((IMG_HEIGHT, IMG_WIDTH, 3), dtype=np.uint8)
        else:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        mask_path_str = row['mask_relative_path']
        if pd.isna(mask_path_str):
            mask = np.zeros((image.shape[0], image.shape[1]), dtype=np.uint8)
        else:
            normalized_mask_path = mask_path_str.replace("\\", "/")
            mask_path = os.path.join(self.base_dir, normalized_mask_path)
            mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
            if mask is None:
                print(f"LỖI NGHIÊM TRỌNG: Vẫn không thể đọc mặt nạ tại: {mask_path}")
                mask = np.zeros((image.shape[0], image.shape[1]), dtype=np.uint8)
            else:
                mask[mask > 0] = 1 
        
        if self.transform:
            augmented = self.transform(image=image, mask=mask)
            image = augmented['image']
            mask = augmented['mask']
            mask = mask.float().unsqueeze(0)
            
        class_label = torch.tensor(class_label, dtype=torch.long)    
        return image, mask, class_label

print("Đã cập nhật định nghĩa lớp BrainMTLDataset.")

Đã cập nhật định nghĩa lớp BrainMTLDataset.


In [6]:
print("Đang tạo các đối tượng Dataset.....")
try:
    train_dataset = BrainMTLDataset(
        manifest_file=TRAIN_MANIFEST,
        base_dir=BASE_DIR,
        transform=train_transforms
    )

    val_dataset = BrainMTLDataset(
        manifest_file=TEST_MANIFEST,
        base_dir=BASE_DIR,
        transform=val_transforms
    )
    print("Đã tạo Dataset thành công.")
except FileNotFoundError as e:
    print(f"LỖI: Không tìm thấy tệp manifest! {e}")

print("\nĐang tạo các đối tượng DataLoader.....")
train_loader = DataLoader(
    dataset=train_dataset,
    batch_size = BATCH_SIZE,
    shuffle=True,
    num_workers=4,
    pin_memory=True
)
val_loader = DataLoader(
    dataset=val_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=4,
    pin_memory=True
)
print("Đã tạo DataLoader thành công.")

Đang tạo các đối tượng Dataset.....
Đã tải ../data/brisc2025/train_clean.csv với 5000 mẫu.
Ánh xạ lớp: {'no_tumor': 0, 'glioma': 1, 'meningioma': 2, 'pituitary': 3}
Đã tải ../data/brisc2025/test_clean.csv với 1000 mẫu.
Ánh xạ lớp: {'no_tumor': 0, 'glioma': 1, 'meningioma': 2, 'pituitary': 3}
Đã tạo Dataset thành công.

Đang tạo các đối tượng DataLoader.....
Đã tạo DataLoader thành công.


In [7]:
print("Đang kiểm tra một batch từ train_loader.....")
try:
    images, masks, labels = next(iter(train_loader))
    images, masks, labels = images.to(DEVICE), masks.to(DEVICE), labels.to(DEVICE)
    print(f"Đã tải batch thành công sang {DEVICE}.")

    print(f"\nShape của batch ảnh (Images): {images.shape}")
    print(f"Shape của batch mặt nạ (Masks): {masks.shape}")
    print(f"Shape của batch nhãn (Labels): {labels.shape}")

    print(f"\nKiểu dữ liệu ảnh: {images.dtype}")
    print(f"Kiểu dữ liệu mặt nạ: {masks.dtype}")
    print(f"Kiểu dữ liệu nhãn: {labels.dtype}")
except Exception as e:
    print(f"Lỗi khi tải batch: {e}")

Đang kiểm tra một batch từ train_loader.....
Đã tải batch thành công sang cuda:0.

Shape của batch ảnh (Images): torch.Size([8, 3, 256, 256])
Shape của batch mặt nạ (Masks): torch.Size([8, 1, 256, 256])
Shape của batch nhãn (Labels): torch.Size([8])

Kiểu dữ liệu ảnh: torch.float32
Kiểu dữ liệu mặt nạ: torch.float32
Kiểu dữ liệu nhãn: torch.int64


In [8]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()

        self.conv_path = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),

            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),

            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels)
        )

        if in_channels != out_channels:
            self.skip_path = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1),
                nn.BatchNorm2d(out_channels)
            )
        else:
            self.skip_path = nn.Identity()
        
        self.final_relu = nn.ReLU(inplace=True)

    def forward(self, x):
        skip_x = self.skip_path(x)
        conv_out = self.conv_path(x)
        output = conv_out + skip_x
        output = self.final_relu(output)
        return output

print("Đã định nghĩa lớp ResidualBlock.")

Đã định nghĩa lớp ResidualBlock.


In [9]:
class BrainMTLModel(nn.Module):
    def __init__(self, in_channels=3, num_classes=4):
        super().__init__()

        self.enc1 = ResidualBlock(in_channels, 64)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.enc2 = ResidualBlock(64, 128)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.enc3 = ResidualBlock(128, 256)
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.enc4 = ResidualBlock(256, 512)
        self.pool4 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.bridge = ResidualBlock(512, 1024)

        self.up4 = nn.ConvTranspose2d(1024, 512, kernel_size=2, stride=2)
        self.dec4 = ResidualBlock(1024, 512)
        self.up3 = nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2)
        self.dec3 = ResidualBlock(512, 256)
        self.up2 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)
        self.dec2 = ResidualBlock(256, 128)
        self.up1 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)
        self.dec1 = ResidualBlock(128, 64)
        self.seg_head = nn.Conv2d(64, 1, kernel_size=1)

        self.gap_enc4 = nn.AdaptiveAvgPool2d(1)
        self.gap_bridge = nn.AdaptiveAvgPool2d(1)
        self.gap_dec4 = nn.AdaptiveAvgPool2d(1)
        total_features = 512 + 1024 + 512
        self.cls_head = nn.Sequential(
            nn.Flatten(),
            nn.Linear(total_features, 256),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        s1 = self.enc1(x)
        s2 = self.enc2(self.pool1(s1))
        s3 = self.enc3(self.pool2(s2))
        s4 = self.enc4(self.pool3(s3))
        b = self.bridge(self.pool4(s4))

        up4 = self.up4(b)
        d4_in = torch.cat([up4, s4], dim=1)
        d4 = self.dec4(d4_in)
        up3 = self.up3(d4)
        d3_in = torch.cat([up3, s3], dim=1)
        d3 = self.dec3(d3_in)
        up2 = self.up2(d3)
        d2_in = torch.cat([up2, s2], dim=1)
        d2 = self.dec2(d2_in)
        up1 = self.up1(d2)
        d1_in = torch.cat([up1, s1], dim=1)
        d1 = self.dec1(d1_in)
        seg_output = self.seg_head(d1)

        f_enc4 = self.gap_enc4(s4).flatten(start_dim=1)
        f_bridge = self.gap_bridge(b).flatten(start_dim=1)
        f_dec4 = self.gap_dec4(d4).flatten(start_dim=1)
        cls_in = torch.cat([f_enc4, f_bridge, f_dec4], dim=1)
        clas_output = self.cls_head(cls_in)

        return seg_output, clas_output

print("Đã định nghĩa lớp BrainMTLModel (U-Net tùy chỉnh + Nhánh Phân loại).")

Đã định nghĩa lớp BrainMTLModel (U-Net tùy chỉnh + Nhánh Phân loại).


In [10]:
print("Đang khởi tạo mô hình (BrainMTLModel).....")
try:
    model = BrainMTLModel(num_classes=NUM_CLASSES).to(DEVICE)
    print(f"Đã khởi tạo mô hình và chuyển sang {DEVICE} thành công.")
except Exception as e:
    print(f"LỖI khi khởi tạo mô hình: {e}")

try:
    dummy_batch = torch.randn(BATCH_SIZE, 3, IMG_HEIGHT, IMG_WIDTH).to(DEVICE)
    print(f"\nĐã tạo batch dữ liệu giả với shape: {dummy_batch.shape}")
    print("Đang cho batch giả di chuyển qua mô hình.....")
    
    with torch.no_grad():
        seg_output, cls_output = model(dummy_batch)
    print("Forward thành công.")

    print("\n----- Kiểm tra kích thước đầu ra -----")

    expected_seg_shape = (BATCH_SIZE, 1, IMG_HEIGHT, IMG_WIDTH)
    print(f" Shape Đầu ra Phân đoạn: {seg_output.shape}")
    if seg_output.shape == expected_seg_shape:
        print(f"CHÍNH XÁC! (Mong đợi: {expected_seg_shape})")
    else:
        print(f"LỖI! (Mong đợi: {expected_seg_shape})")

    expected_cls_shape = (BATCH_SIZE, NUM_CLASSES)
    print(f"\nShape Đầu ra Phân loại: {cls_output.shape}")
    if cls_output.shape == expected_cls_shape:
        print(f"CHÍNH XÁC! (Mong đợi: {expected_cls_shape})")
    else:
        print(f"LỖI! (Mong đợi: {expected_cls_shape})")  
except Exception as e:
    print(f"\nLỖI trong quá trình kiểm tra mô hình: {e}")

Đang khởi tạo mô hình (BrainMTLModel).....
Đã khởi tạo mô hình và chuyển sang cuda:0 thành công.

Đã tạo batch dữ liệu giả với shape: torch.Size([8, 3, 256, 256])
Đang cho batch giả di chuyển qua mô hình.....
Forward thành công.

----- Kiểm tra kích thước đầu ra -----
 Shape Đầu ra Phân đoạn: torch.Size([8, 1, 256, 256])
CHÍNH XÁC! (Mong đợi: (8, 1, 256, 256))

Shape Đầu ra Phân loại: torch.Size([8, 4])
CHÍNH XÁC! (Mong đợi: (8, 4))


In [11]:
print("Đang tính toán trọng số cho các lớp phân loại.....")
try:
    df = train_dataset.df
    class_counts = df['tumor_label'].value_counts().sort_index()
    sorted_labels = sorted(train_dataset.labels_map.keys(), key=lambda k: train_dataset.labels_map[k])
    class_counts = class_counts.reindex(sorted_labels).values

    total_samples = len(df)
    num_classes = len(class_counts)
    class_weights = total_samples / (num_classes * class_counts)
    class_weights_tensor = torch.tensor(class_weights, dtype=torch.float32).to(DEVICE)

    print(f"Số lượng mẫu các lớp: {class_counts}")
    print(f"Trọng số tính toán: {class_weights_tensor}")
    print("Đã tính xong trọng số phân loại.")
except Exception as e:
    print(f"LỖI khi tính toán trọng số: {e}")
    print("Sẽ sử dụng trọng số mặc định (None).")
    class_weights_tensor = None

Đang tính toán trọng số cho các lớp phân loại.....
Số lượng mẫu các lớp: [1067 1147 1329 1457]
Trọng số tính toán: tensor([1.1715, 1.0898, 0.9406, 0.8579], device='cuda:0')
Đã tính xong trọng số phân loại.


In [12]:
class DiceLoss(nn.Module):
    def __init__(self, smooth=1e-6):
        super(DiceLoss, self).__init__()
        self.smooth = smooth

    def forward(self, logits, targets):
        probs = torch.sigmoid(logits)
        probs = probs.view(-1)
        targets = targets.view(-1)
        intersection = (probs * targets).sum()
        dice_coeff = (2. * intersection + self.smooth) / (probs.sum() + targets.sum() + self.smooth)
        return 1.0 - dice_coeff

print("Đã định nghĩa lớp (class) DiceLoss.")

print("Đang khởi tạo các đối tượng hàm loss...")
seg_loss_bce = nn.BCEWithLogitsLoss().to(DEVICE)
seg_loss_dice = DiceLoss().to(DEVICE)
cls_loss_fn = nn.CrossEntropyLoss(weight=class_weights_tensor).to(DEVICE)
print("Đã khởi tạo tất cả các hàm loss.")

optimizer = optim.Adam(
    model.parameters(), 
    lr=LEARNING_RATE
)
scaler = GradScaler(enabled=(DEVICE.type == 'cuda'))
print(f"Đã khởi tạo Optimizer: Adam (Learning Rate: {LEARNING_RATE})")
print(f"Đã khởi tạo GradScaler (Enabled: {DEVICE.type == 'cuda'}).")

Đã định nghĩa lớp (class) DiceLoss.
Đang khởi tạo các đối tượng hàm loss...
Đã khởi tạo tất cả các hàm loss.
Đã khởi tạo Optimizer: Adam (Learning Rate: 0.001)
Đã khởi tạo GradScaler (Enabled: True).


/tmp/ipykernel_8215/1223927299.py:26: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler(enabled=(DEVICE.type == 'cuda'))


In [13]:
def check_dice_score(preds, targets, smooth=1e-6):
    preds = torch.sigmoid(preds)
    preds = (preds > 0.5).float()
    preds = preds.view(-1)
    targets = targets.view(-1)
    intersection = (preds * targets).sum()
    dice = (2. * intersection + smooth) / (preds.sum() + targets.sum() + smooth)
    return dice

In [14]:
def train_fn(loader, model, optimizer, cls_loss_fn, seg_loss_bce, seg_loss_dice, scaler):
    loop = tqdm(loader, desc="Training Epoch", leave=True)
    avg_total_loss = 0.0
    avg_cls_loss = 0.0
    avg_seg_loss = 0.0
    
    all_cls_correct = 0
    all_samples = 0
    all_dice_scores = []

    model.train()
    for batch_idx, (images, masks, labels) in enumerate(loop):
        images = images.to(DEVICE)
        masks = masks.to(DEVICE)
        labels = labels.to(DEVICE)

        with autocast(enabled=(DEVICE.type == 'cuda')):
            seg_pred, cls_pred = model(images)
            cls_loss = cls_loss_fn(cls_pred, labels)
            tumor_indices = (labels != 0)
            if tumor_indices.sum() > 0:
                seg_pred_filtered = seg_pred[tumor_indices]
                masks_filtered = masks[tumor_indices]
                loss_bce = seg_loss_bce(seg_pred_filtered, masks_filtered)
                loss_dice = seg_loss_dice(seg_pred_filtered, masks_filtered)
                total_seg_loss = loss_bce + loss_dice
                
                dice_score = check_dice_score(seg_pred_filtered, masks_filtered)
                all_dice_scores.append(dice_score.item())
            else:
                total_seg_loss = torch.tensor(0.0).to(DEVICE)
            total_loss = cls_loss + total_seg_loss
        
        optimizer.zero_grad()
        scaler.scale(total_loss).backward()
        scaler.step(optimizer)
        scaler.update()
        
        _, predictions = cls_pred.max(1)
        all_cls_correct += (predictions == labels).sum().item()
        all_samples += labels.size(0)
        
        avg_total_loss += total_loss.item()
        avg_cls_loss += cls_loss.item()
        avg_seg_loss += total_seg_loss.item() if isinstance(total_seg_loss, torch.Tensor) else total_seg_loss
        
        loop.set_postfix(
            total_loss=f"{total_loss.item():.4f}",
            cls_loss=f"{cls_loss.item():.4f}",
            seg_loss=f"{total_seg_loss.item() if isinstance(total_seg_loss, torch.Tensor) else 0.0:.4f}"
        )
        
    len_loader = len(loader)
    final_avg_total = avg_total_loss / len_loader
    final_avg_cls = avg_cls_loss / len_loader
    final_avg_seg = avg_seg_loss / len_loader
    final_cls_acc = (all_cls_correct / all_samples) * 100
    final_dice_score = (sum(all_dice_scores) / len(all_dice_scores)) if len(all_dice_scores) > 0 else 0.0
    
    print(f"\n--- Kết thúc Epoch Huấn luyện ---")
    print(f"  Avg Total Loss: {final_avg_total:.4f}")
    print(f"  Avg Cls Loss:   {final_avg_cls:.4f}  |  Avg Cls Acc:   {final_cls_acc:.2f}%")
    print(f"  Avg Seg Loss:   {final_avg_seg:.4f}  |  Avg Seg Dice: {final_dice_score:.4f}")
    
print("Đã định nghĩa hàm train_fn.")

Đã định nghĩa hàm train_fn.


In [15]:
def validate_fn(loader, model, cls_loss_fn, seg_loss_bce, seg_loss_dice):
    loop = tqdm(loader, desc="Validating", leave=True)
    
    model.eval()
    avg_total_loss = 0.0
    avg_cls_loss = 0.0
    avg_seg_loss = 0.0
    
    all_cls_correct = 0
    all_samples = 0
    all_dice_scores = []
    
    with torch.no_grad():
        for batch_idx, (images, masks, labels) in enumerate(loop):
            images = images.to(DEVICE)
            masks = masks.to(DEVICE)
            labels = labels.to(DEVICE)
            
            with autocast(enabled=(DEVICE.type == 'cuda')):
                seg_pred, cls_pred = model(images)
                cls_loss = cls_loss_fn(cls_pred, labels)
                
                tumor_indices = (labels != 0)
                if tumor_indices.sum() > 0:
                    seg_pred_filtered = seg_pred[tumor_indices]
                    masks_filtered = masks[tumor_indices]
                    loss_bce = seg_loss_bce(seg_pred_filtered, masks_filtered)
                    loss_dice = seg_loss_dice(seg_pred_filtered, masks_filtered)
                    total_seg_loss = loss_bce + loss_dice
                else:
                    total_seg_loss = torch.tensor(0.0).to(DEVICE)
                    
                total_loss = cls_loss + total_seg_loss

            _, predictions = cls_pred.max(1)
            all_cls_correct += (predictions == labels).sum().item()
            all_samples += labels.size(0)
            
            if tumor_indices.sum() > 0:
                dice_score = check_dice_score(seg_pred_filtered, masks_filtered)
                all_dice_scores.append(dice_score.item())

            avg_total_loss += total_loss.item()
            avg_cls_loss += cls_loss.item()
            avg_seg_loss += total_seg_loss.item() if isinstance(total_seg_loss, torch.Tensor) else total_seg_loss

    model.train() 
    
    len_loader = len(loader)
    final_avg_total = avg_total_loss / len_loader
    final_avg_cls = avg_cls_loss / len_loader
    final_avg_seg = avg_seg_loss / len_loader
    
    final_cls_acc = (all_cls_correct / all_samples) * 100
    final_dice_score = (sum(all_dice_scores) / len(all_dice_scores)) if len(all_dice_scores) > 0 else 0.0

    print(f"\n--- KẾT QUẢ VALIDATION ---")
    print(f"  Avg Total Loss: {final_avg_total:.4f}")
    print(f"  Avg Cls Loss:   {final_avg_cls:.4f}  |  Cls Accuracy:   {final_cls_acc:.2f}%")
    print(f"  Avg Seg Loss:   {final_avg_seg:.4f}  |  Dice Score (Seg): {final_dice_score:.4f}")
    print("----------------------------")

    return final_avg_total, final_cls_acc, final_dice_score

print("Đã định nghĩa hàm validate_fn.")

Đã định nghĩa hàm validate_fn.


In [16]:
class EarlyStopper:
    def __init__(self, patience=7, min_delta=0.0001, save_path="best_mtl_model.pth"):
        self.patience = patience
        self.min_delta = min_delta
        self.save_path = save_path
        self.counter = 0
        self.best_loss = np.inf
        self.early_stop = False
        
        save_dir = os.path.dirname(self.save_path)
        if save_dir and not os.path.exists(save_dir):
            print(f"Tạo thư mục: {save_dir}")
            os.makedirs(save_dir, exist_ok=True)

    def __call__(self, val_loss, model):
        if val_loss < self.best_loss - self.min_delta:
            print(f"Val loss cải thiện ({self.best_loss:.4f} --> {val_loss:.4f}). Đang lưu mô hình...")
            self.best_loss = val_loss
            self.counter = 0
            torch.save(model.state_dict(), self.save_path)
        else:
            self.counter += 1
            print(f"Val loss không cải thiện. Bộ đếm: {self.counter} / {self.patience}")
            if self.counter >= self.patience:
                print(f"--- Dừng Sớm (Early Stopping) kích hoạt ---")
                self.early_stop = True

scheduler = ReduceLROnPlateau(
    optimizer, 
    mode='min',
    factor=0.1,
    patience=5
)

early_stopper = EarlyStopper(
    patience=10, 
    save_path=os.path.join(MODEL_SAVE_PATH, "best_mtl_model_early_stop.pth")
)

print(f"Đã khởi tạo EarlyStopper.")
print(f"Đã khởi tạo Hẹn giờ (Scheduler) và Dừng Sớm (EarlyStopper).")

Đã khởi tạo EarlyStopper.
Đã khởi tạo Hẹn giờ (Scheduler) và Dừng Sớm (EarlyStopper).


In [17]:
print("========== BẮT ĐẦU HUẤN LUYỆN ==========")

for epoch in range(EPOCHS):
    print(f"\n======= Epoch {epoch + 1} / {EPOCHS} =======")
    
    train_fn(
        train_loader, 
        model, 
        optimizer, 
        cls_loss_fn, 
        seg_loss_bce, 
        seg_loss_dice, 
        scaler
    )
    
    val_loss, val_acc, val_dice = validate_fn(
        val_loader, 
        model, 
        cls_loss_fn, 
        seg_loss_bce, 
        seg_loss_dice
    )
    
    scheduler.step(val_loss)
    
    early_stopper(val_loss, model)
    
    if early_stopper.early_stop:
        print("Dừng huấn luyện sớm do val_loss không cải thiện.")
        break

print("\n========== HUẤN LUYỆN HOÀN TẤT ==========")
print(f"Mô hình tốt nhất đã được lưu tại: {early_stopper.save_path}")
print(f"Validation Loss tốt nhất đạt được: {early_stopper.best_loss:.4f}")

========== BẮT ĐẦU HUẤN LUYỆN ==========

======= Epoch 1 / 60 =======


Training Epoch:   0%|          | 0/625 [00:00<?, ?it/s]/tmp/ipykernel_8215/921836045.py:17: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(enabled=(DEVICE.type == 'cuda')):
Training Epoch: 100%|██████████| 625/625 [19:30<00:00,  1.87s/it, cls_loss=1.1569, seg_loss=0.6065, total_loss=1.7634]



--- Kết thúc Epoch Huấn luyện ---
  Avg Total Loss: 1.6312
  Avg Cls Loss:   0.9693  |  Avg Cls Acc:   60.92%
  Avg Seg Loss:   0.6619  |  Avg Seg Dice: 0.4565


Validating:   0%|          | 0/125 [00:00<?, ?it/s]/tmp/ipykernel_8215/497464996.py:19: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(enabled=(DEVICE.type == 'cuda')):
Validating: 100%|██████████| 125/125 [00:22<00:00,  5.49it/s]



--- KẾT QUẢ VALIDATION ---
  Avg Total Loss: 3.3970
  Avg Cls Loss:   2.8314  |  Cls Accuracy:   49.30%
  Avg Seg Loss:   0.5656  |  Dice Score (Seg): 0.4833
----------------------------
Val loss cải thiện (inf --> 3.3970). Đang lưu mô hình...

======= Epoch 2 / 60 =======


...


--- KẾT QUẢ VALIDATION ---
  Avg Total Loss: 0.2407
  Avg Cls Loss:   0.0769  |  Cls Accuracy:   99.30%
  Avg Seg Loss:   0.1639  |  Dice Score (Seg): 0.8425
----------------------------
Val loss không cải thiện. Bộ đếm: 9 / 10

======= Epoch 39 / 60 =======


Training Epoch: 100%|██████████| 625/625 [19:25<00:00,  1.86s/it, cls_loss=0.0003, seg_loss=0.2685, total_loss=0.2689]



--- Kết thúc Epoch Huấn luyện ---
  Avg Total Loss: 0.1825
  Avg Cls Loss:   0.0127  |  Avg Cls Acc:   99.50%
  Avg Seg Loss:   0.1698  |  Avg Seg Dice: 0.8571


Validating: 100%|██████████| 125/125 [00:21<00:00,  5.93it/s]


--- KẾT QUẢ VALIDATION ---
  Avg Total Loss: 0.2423
  Avg Cls Loss:   0.0789  |  Cls Accuracy:   99.20%
  Avg Seg Loss:   0.1635  |  Dice Score (Seg): 0.8429
----------------------------
Val loss không cải thiện. Bộ đếm: 10 / 10
--- Dừng Sớm (Early Stopping) kích hoạt ---
Dừng huấn luyện sớm do val_loss không cải thiện.

========== HUẤN LUYỆN HOÀN TẤT ==========
Mô hình tốt nhất đã được lưu tại: ../models/best_mtl_model_early_stop.pth
Validation Loss tốt nhất đạt được: 0.2342
